# Loan Level Calculator

The class is created for calculating loan information. Given the notional, interest rate, terms and the period you wannt to check, the calculator can return monthlypmt, makePrincipalPayment, makeInterestPayment, notionalBalance.

LoanInfo: Initialized with loan notional and rate. 

StandardTranche derived from parent class LoanInfo. 
This child class keeps track of all interest payments and all principal payments made to it, and at what time period, implements the following methods:
* monthlypmt: return the monthly payment
* makePrincipalPayment: record a principal payment for the current object time period. 
* makeInterestPayment: record an interest payment for the current object time period. 
* notionalBalance: return the amount of notional still owed to the tranche for the current time period (after any payments made). 

Calculation theory:

periodic loan payment: pmt = rP/(1-(1+r)^(-N) = rP(1+r)^N/((1+r)^N - 1)
remaing balance: bal = P(1+r)^n - pmt[((1+r)^n-1)/r]

where:
* P = Principal
* r = Monthly interest rate
* N = Term of loan
* n = Number of payments made
* pmt = Monthly payment
* bal = Balance of Loan

In [14]:
# Parent class having notional and rate
class LoanInfo(object):
    def __init__(self, notional, rate):
        self._notional = notional
        self._rate = rate
    
    @property
    def notional(self):
        return self._notional
    
    @notional.setter
    def notional(self, inotional):
        self._notional = inotional
    
    @property
    def rate(self):
        return self._rate
    
    @rate.setter
    def rate(self, irate):
        self._rate = irate

In [15]:
# Child Class of LoanInfo, which can calculate the monthly payment, monthly interest payment, and balance lefe
class StandardTranche(LoanInfo):
    def __init__(self, notional, rate, term, period):
        super().__init__(notional, rate)
#         current time period
        self._period = period
#         total years to pay the mortgage
        self._term = term
        
    @property
    def period(self):
        return self._period
    
    @period.setter
    def period(self, iperiod):
        self._period = iperiod
    
    @property
    def term(self):
        return self._term
    
    @term.setter
    def term(self, iterm):
        self._term = iterm
    
    @classmethod
    def monthlypmt(cls, notional, rate, term):
        if notional == 0:
            return 'The current notional is 0, payment not accepted'
        else:
        #  P0 * r * (1 + r)^n
            loanAmount = notional * (rate/12) * ((1 + rate/12) ** (term*12))
        # (1 + r)^n -1    
            compoundRate = (1 + rate/12) ** (term*12) - 1
        
            monPMT = loanAmount/compoundRate
        
        return monPMT 

    
    @classmethod
    # amount of notional still owned to tranche
    def notionalBalance(cls, notional, rate, term, period):
        termOfbalance = period - 1
        # if on term 1, the balance is face value
        if period <= 1:
            balanceleft = notional

        else:
            # initial balance is: p(1+r)^n
            # balanceC = self._face * (1 + self._rate/12) ** termOfbalance
            balanceC = notional * (1 + rate/12) ** termOfbalance

            # pmt discounted back to current term: pmt(((1+r)^n - 1) / r)
            # use delegate method from class-level
            currentPMT = StandardTranche.monthlypmt(notional, rate, term) * (((1 + rate/12) ** termOfbalance - 1) / (rate/12))

            # balance left on given term
            balanceleft = balanceC - currentPMT

        return balanceleft
    
        
    def makePrincipalPayment(self):
        # currentPrincipal = current pmt - current interest
        currentPrincipal =  StandardTranche.monthlypmt(self._notional, self._rate, self._term) - self.makeInterestPayment()
        return currentPrincipal
    
    def makeInterestPayment(self):
        # make period less 1
        pastperiod = self._period - 1
        # the principal = rate/12 * balance on last period
        currentInterest = (self._rate/12) *  StandardTranche.notionalBalance(self._notional, self._rate, self._term, pastperiod)
        return currentInterest
       
        


## Test the class

#### 1. Test if loan information can be shown

In [27]:
loan = LoanInfo(10000, 0.05)

In [28]:
loan.notional

10000

In [29]:
loan.rate

0.05

#### 2. Test the functions in child class


For example, a loan with $100000, annual rate of 0.05, 30 years. 

We can check loan information on the period of 100：
1. monthly pmt for every month is $536.82, 

2. notional balance left is $85313.35, 

3. an interest payment of $356.22, 

4. the principal payment of $180.6.

In [30]:
# Define a loan with principle of 100000, rate of 0.05, term of loan of 30 years, period we are checking 100.
test = StandardTranche(100000, 0.05, 30, 100)

In [31]:
test.monthlypmt(100000, 0.05, 30)

536.8216230121398

In [32]:
test.notionalBalance(100000, 0.05, 30, 100)

85313.34831405093

In [33]:
test.makeInterestPayment()

356.22477152308323

In [34]:
test.makePrincipalPayment()

180.59685148905652